Background
 - The team has been asked to evaluate the recent revenue performance for a health insurance product. We are particularly interested in how long customers are keeping these insurance policies

Data description
 - The data consists of transactions from 7/1/2018 to 6/7/2019
 - It includes:
    - Customer ID
    - Membership Start Date – start of policy
    -Transaction Date – date of payment
    - Payment Type (Annual Fee or Monthly Fee)
    - Revenue – amount of payment received

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')

In [2]:
data=pd.read_csv("dataset_healthinsurance.csv")

In [3]:
df = data.copy()

In [4]:
df

,Customer ID,Membership Start Date,Transaction Date,Payment Type,Revenue
0,386379,2019-02-19,2019-02-18,Annual Fee,285.9
1,386379,2019-02-19,2019-02-18,Monthly Fee,26.5
2,386379,2019-02-19,2019-03-19,Monthly Fee,26.5
3,386379,2019-02-19,2019-04-19,Monthly Fee,26.5
4,386379,2019-02-19,2019-05-19,Monthly Fee,26.5
...,...,...,...,...,...
49460,397988,2019-04-02,2019-04-01,Annual Fee,331.9
49461,397988,2019-04-02,2019-04-01,Monthly Fee,26.5
49462,398359,2019-04-02,2019-04-02,Annual Fee,517.0
49463,398359,2019-04-02,2019-04-02,Monthly Fee,26.5


Question 1: Which month had the highest revenue (by "Transaction Date")? What was the percentage of revenue for that month by payment type?

In [5]:
# Extract the month and year from the 'Transaction Date'
df['Transaction_Month'] = pd.to_datetime(df['Transaction Date']).dt.to_period('M')

# Group by the extracted month and sum the revenue
monthly_revenue = df.groupby('Transaction_Month')['Revenue'].sum()

# Find the month with the highest revenue
max_revenue_month = monthly_revenue.idxmax()
max_revenue_value = monthly_revenue.max()

# Filter the data for that month and group by Payment Type
payment_type_revenue = df[df['Transaction_Month'] == max_revenue_month].groupby('Payment Type')['Revenue'].sum()

# Calculate the percentage of revenue by payment type for that month
payment_type_percentage = (payment_type_revenue / max_revenue_value) * 100

print(f"The month with highest revenue is: {max_revenue_month}")
print(payment_type_percentage)

The month with highest revenue is: 2018-12
Payment Type
Annual Fee     91.572934
Monthly Fee     8.427066
Name: Revenue, dtype: float64


Question 2: How many members started their memberships in December 2018 (by "Membership Start Date")?

In [6]:
# Convert 'Membership Start Date' to datetime format
df['Membership Start Date'] = pd.to_datetime(df['Membership Start Date'])

# Filter the data to only include entries with a "Membership Start Date" in December 2018
december_2018_members = df[df['Membership Start Date'].dt.month == 12][df['Membership Start Date'].dt.year == 2018]

# Count the unique "Customer IDs" for this filtered data to get the number of new members in December 2018
unique_members_december_2018 = december_2018_members['Customer ID'].nunique()

print(f"Number of members who started their memberships in December 2018: {unique_members_december_2018}")

Number of members who started their memberships in December 2018: 1254


Question 3: On average, for how many months did members who started their memberships in July 2018 continue to pay the monthly fee?

In [7]:
# Convert relevant columns to datetime format
df['Membership Start Date'] = pd.to_datetime(df['Membership Start Date'])
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])

# Filter for members who started their memberships in July 2018
july_2018_members = df[df['Membership Start Date'].dt.month == 7][df['Membership Start Date'].dt.year == 2018]['Customer ID'].unique()

# Filter transactions of these members which are of type 'Monthly Fee'
july_2018_member_transactions = df[df['Customer ID'].isin(july_2018_members) & (df['Payment Type'] == 'Monthly Fee')]

# Count the monthly transactions for each member
transaction_counts = july_2018_member_transactions.groupby('Customer ID').size()

# Calculate the average number of months
average_months = transaction_counts.mean()

print(f"On average, members who started their memberships in July 2018 continued to pay the monthly fee for {average_months:.2f} months.")

On average, members who started their memberships in July 2018 continued to pay the monthly fee for 4.92 months.


Question 4: Assume that transactions for June 2019 are incomplete and that transactions beyond June 2019 are unobserved. What type of statistical or machine learning modeling technique(s) would you use to build a predictive model of customer retention?

To answer this question we'll use a logistic regression model.

We'll create a binary column called "Churned", where 1 indicates the customer has churned and 0 indicates they have not. This could be determined by checking if there's a lack of a transaction for a certain period (for example, if a customer hasn't made a transaction for 3 months).

We'll create features that could be predictive of churn, such as:
 - Duration of membership
 - Total revenue from a customer
 - Average transaction amount
 - Time since the last transaction
 - Number of transactions made by the customer

To develop and evaluate the model, we'll split the data into a training set and a test set and fit a logistic regression model on the training data.

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder


# Derive the target variable (this is a hypothetical method; real method would depend on specific criteria)
df['Churned'] = df.groupby('Customer ID')['Transaction Date'].transform('max')
df['Churned'] = ((pd.to_datetime('6/7/2019') - df['Churned']).dt.days > 90).astype(int)  # Churned if no transaction in last 3 months

# Convert Payment Type into numeric using one-hot encoding
encoder = OneHotEncoder(drop='first')
payment_encoded = encoder.fit_transform(data[['Payment Type']])
payment_df = pd.DataFrame(payment_encoded.toarray(), columns=encoder.get_feature_names(['Payment Type']))

df = pd.concat([df, payment_df], axis=1)  # .drop(columns='Payment Type')

# Create other features (hypothetical; real methods would depend on the specific criteria and domain knowledge)
df['Duration of membership'] = (pd.to_datetime(df['Transaction Date']) - pd.to_datetime(df['Membership Start Date'])).dt.days

# Split the data
X = df[['Duration of membership', 'Payment Type_Monthly Fee', 'Revenue']]  # and any other features
y = df['Churned']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on test set (just as an example)
y_pred = model.predict(X_test)

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Predict probabilities for the positive class
y_prob = model.predict_proba(X_test)[:,1]

# Compute metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)


print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")


Accuracy: 0.67
Precision: 0.51


Question 5: How many customers paid the Monthly Fee in May 2019? Predict the Membership End Date for these customers.

We'll filter the data for May 2019 to find customers who paid the Monthly Fee. As we can't make any assumptions about membership end dates based on June 2019 transactions, and we can't employ a machine learning model without relevant training data including Membership End Date, we can take an approach based on historical data.

We'll calculate the average membership duration for each customer up to May 2019, and use this average to project an expected membership end date for the customers identified in the previous step.

In [10]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
df['Membership Start Date'] = pd.to_datetime(df['Membership Start Date'])

# Filter for transactions in May 2019 with Monthly Fee
may_2019_payments = df[(df['Transaction Date'].dt.month == 5) & 
                       (df['Transaction Date'].dt.year == 2019) & 
                       (df['Payment Type'] == 'Monthly Fee')]

customers_count = may_2019_payments['Customer ID'].nunique()
print(f"There are {customers_count} customers who paid the Monthly Fee in May 2019.")

# Calculate the average membership duration up to May 2019 for each customer
df_before_june_2019 = df[df['Transaction Date'] <= '2019-05-31']
df_before_june_2019['Duration'] = (df_before_june_2019['Transaction Date'] - df_before_june_2019['Membership Start Date']).dt.days
avg_durations = df_before_june_2019.groupby('Customer ID')['Duration'].mean()

# Predict the end date using the average duration
may_2019_payments = may_2019_payments.set_index('Customer ID')
may_2019_payments['Average Duration'] = avg_durations
may_2019_payments['Predicted End Date'] = may_2019_payments['Transaction Date'] + pd.to_timedelta(may_2019_payments['Average Duration'], unit='d')

print(may_2019_payments[['Predicted End Date']])

There are 4533 customers who paid the Monthly Fee in May 2019.
                       Predicted End Date
Customer ID                              
386379      2019-06-22 19:12:00.000000000
385532      2019-06-18 19:12:00.000000000
382351      2019-06-05 19:12:00.000000000
363517      2019-06-11 00:00:00.000000000
362304      2019-07-05 17:08:34.285689600
...                                   ...
397958      2019-05-18 00:00:00.000000000
397965      2019-05-11 07:59:59.999971200
397969      2019-05-11 07:59:59.999971200
397983      2019-05-11 07:59:59.999971200
398359      2019-05-12 00:00:00.000000000

[4585 rows x 1 columns]


This approach assumes that a customer's past behavior is indicative of their future behavior. This might not always be the case. We're relying on the mean duration, which can be sensitive to outliers. Median or mode could be alternative metrics to consider. We're considering all past transactions, but it might be useful to give more weight to recent transactions.